In [1]:
import os
import random

import cv2
import face_recognition
import numpy as np
from tqdm import tqdm_notebook as tqdm

from constants import FRAMES_IN_SEQ, FACE_SHAPE
from helpers import get_face, get_path_and_indexes

In [2]:
files = [item for item in os.listdir(".") if "mp4" in item]
files

['1_real.mp4',
 '2_real.mp4',
 '1_fake.mp4',
 '2_fake.mp4',
 '1_fake2.mp4',
 '2_real2.mp4']

In [5]:
train_idx = 0
val_idx = 0
test_idx = 0
for file in files:
    gt = "fake" if "fake" in file else "real"
    flip = True if gt == "fake" else False  # fake frames need to be mirrored
    v = cv2.VideoCapture(file)
    w = int(v.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(v.get(cv2.CAP_PROP_FRAME_HEIGHT))
    num_frames = v.get(cv2.CAP_PROP_FRAME_COUNT)
    frames_seq = np.zeros((FRAMES_IN_SEQ, h, w, 3), dtype=np.uint8)
    # prepopulating array with all but one frames
    for i in range(FRAMES_IN_SEQ - 1):
        ret, frame = v.read()
        frame = frame[..., ::-1]  # BGR2RGB
        if flip:
            frame = cv2.flip(frame, 1)
        frames_seq[i] = frame
    for i in tqdm(range(FRAMES_IN_SEQ - 1, int(num_frames - FRAMES_IN_SEQ))):
        ret, frame = v.read()
        frame = frame[..., ::-1]  # BGR2RGB
        if flip:
            frame = cv2.flip(frame, 1)
        frames_seq[-1] = frame
        found, faces = get_face(frames_seq, FACE_SHAPE)
        if found:
            # forward order
            out_path, train_idx, val_idx, test_idx = get_path_and_indexes(gt, train_idx, val_idx, test_idx)
            np.savez(out_path, seq=faces)
            # reverse order for data augmentation
            out_path, train_idx, val_idx, test_idx = get_path_and_indexes(gt, train_idx, val_idx, test_idx)
            np.savez(out_path, seq=np.flip(faces, 0))
        # rollling frames, so that the first one will become the last one and will be overwritten
        frames_seq = np.roll(frames_seq, FRAMES_IN_SEQ - 1, 0)